<a href="https://colab.research.google.com/github/Feranie/Hierarchical-Classification-Project/blob/main/hierarchical_global_model_naives_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importação das bibliotecas necessárias
import numpy as np  # Biblioteca para operações numéricas e arrays
import pandas as pd  # Biblioteca para manipulação de dados (não usada neste código)
from collections import defaultdict, Counter  # Estruturas de dados para contagem e agrupamento
from typing import List, Set, Tuple, Dict  # Tipagem para melhor documentação do código
import math  # Biblioteca matemática básica (não usada neste código)
from sklearn.model_selection import train_test_split, KFold  # Funções para divisão de dados

class ArffReader:
    """Classe para ler arquivos no formato ARFF (Attribute-Relation File Format)"""

    def __init__(self):
        """Construtor da classe - inicializa listas vazias"""
        self.attributes = []  # Lista para armazenar os nomes dos atributos
        self.data = []  # Lista para armazenar os dados/instâncias

    def read_arff(self, input_file):
        """Método para ler e processar um arquivo ARFF"""
        reading_data = False  # Flag para indicar se estamos lendo a seção de dados
        try:
            # Abre o arquivo em modo leitura com codificação UTF-8
            with open(input_file, 'r', encoding='utf-8') as file:
                # Itera sobre cada linha do arquivo
                for line in file:
                    line = line.strip()  # Remove espaços em branco no início e fim da linha
                    # Pula linhas vazias ou linhas de comentário (começam com %)
                    if not line or line.startswith('%'):
                        continue
                    # Verifica se a linha define um atributo
                    if '@attribute' in line.lower():
                        attr_name = line.split(' ')[1]  # Extrai o nome do atributo (segunda palavra)
                        self.attributes.append(attr_name)  # Adiciona o atributo à lista
                        continue
                    # Verifica se chegou na seção de dados
                    if '@data' in line.lower():
                        reading_data = True  # Marca que começou a seção de dados
                        continue
                    # Se estamos na seção de dados, processa a linha
                    if reading_data:
                        self.data.append(line.split(','))  # Divide a linha por vírgulas e adiciona aos dados
            # Exibe informações sobre o arquivo lido
            print(f"Total d'attributs : {len(self.attributes)}")  # Mostra número total de atributos
            print(f"Total d'instances : {len(self.data)}")  # Mostra número total de instâncias

            # # Imprime dados das listas
            # print("Lista de Atributos")
            # print(self.attributes)
            # print("Dados:")
            # for line in self.data:
            #   print(line)
        except Exception as e:
            # Em caso de erro, exibe mensagem e relança a exceção
            print(f"Erreur lors de la lecture du fichier : {str(e)}")
            raise

class HierarchicalNaiveBayes:
    """Classe principal que implementa o algoritmo Naive Bayes Hierárquico"""

    def __init__(self, alpha: float = 1.0):
        """Construtor - inicializa o modelo com parâmetro de suavização"""
        self.alpha = alpha  # Parâmetro para suavização de Laplace
        self.classes = []  # Lista das classes (folhas da hierarquia)
        self.feature_probs = []  # Probabilidades dos atributos por classe
        self.prior_prob = {}  # Probabilidades a priori das classes
        self.n_values_per_att = {}  # Número de valores distintos por atributo
        self.n_class_occurrences = {}  # Contagem de ocorrências por classe
        self.descendants = {}  # Dicionário de descendentes para cada classe
        self.ancestors = {}  # Dicionário de ancestrais para cada classe
        self.all_possible_classes = set()  # Conjunto de todas as classes possíveis na hierarquia

    def gera_hierarquia_completa(self, classes):
        """Gera a hierarquia completa a partir das classes folha"""
        hierarquia_completa = []  # Lista para armazenar todas as classes da hierarquia
        # Para cada classe folha
        for classe in classes:
            parts = classe.split('.')  # Divide o nome da classe pelos pontos
            # Gera todas as classes intermediárias (do mais específico ao mais geral)
            for i in range(len(parts), 0, -1):
                current_class = '.'.join(parts[:i])  # Reconstrói a classe até o nível i
                # Adiciona à hierarquia se ainda não existe
                if current_class not in hierarquia_completa:
                    hierarquia_completa.append(current_class)
                    self.all_possible_classes.add(current_class)  # Adiciona ao conjunto de todas as classes
        return hierarquia_completa

    def gera_ancestrais(self, classes):
        """Gera os ancestrais para cada classe"""
        ancestrais = {}  # Dicionário para mapear classe -> lista de ancestrais
        # Para cada classe
        for c1 in classes:
            ancestrais[c1] = []  # Inicializa lista vazia de ancestrais
            parts = c1.split('.')  # Divide o nome da classe pelos pontos
            current = ""  # String para construir o nome do ancestral
            # Constrói a lista de ancestrais do mais geral ao mais específico
            for i, part in enumerate(parts):
                if i == 0:
                    current = part  # Primeiro nível
                else:
                    current = f"{current}.{part}"  # Adiciona o próximo nível
                # Adiciona como ancestral se não for a própria classe
                if current != c1:
                    ancestrais[c1].append(current)
        return ancestrais

    def gera_descendentes(self, classes):
        """Gera os descendentes para cada classe"""
        descendentes = {}  # Dicionário para mapear classe -> lista de descendentes
        # Para cada classe
        for c1 in classes:
            descendentes[c1] = []  # Inicializa lista vazia de descendentes
            # Verifica todas as outras classes
            for c2 in classes:
                # c1 é ancestral de c2 se c2 começa com c1 seguido de um ponto
                if c2.startswith(c1 + '.'):
                    descendentes[c1].append(c2)  # Adiciona c2 como descendente de c1
        return descendentes

    def calculate_usefulness(self):
        """Calcula a utilidade de cada classe baseada no tamanho da subárvore"""
        a_constant = 1  # Constante para o cálculo
        max_value = 0  # Valor máximo encontrado

        # Primeira passagem: encontra o valor máximo de a(ci) * log2(treesize(ci))
        for c in self.classes:
            treesize_ci = 1 + len(self.descendants.get(c, []))  # Tamanho da subárvore (+1 para incluir a própria classe)
            a_ci = a_constant if self.prior_prob.get(c, 0) > 0 else 0  # Valor de a(ci) baseado na probabilidade a priori
            value = a_ci * np.log2(treesize_ci)  # Calcula o valor
            if value > max_value:
                max_value = value  # Atualiza o valor máximo

        # Segunda passagem: calcula a utilidade de cada classe
        usefulness = {}  # Dicionário para armazenar a utilidade de cada classe
        for c in self.classes:
            treesize_ci = 1 + len(self.descendants.get(c, []))  # Tamanho da subárvore
            a_ci = a_constant if self.prior_prob.get(c, 0) > 0 else 0  # Valor de a(ci)
            usefulness_ci = 1.0  # Valor padrão de utilidade
            # Calcula a utilidade normalizada
            if max_value > 0:
                usefulness_ci = 1 - (a_ci * np.log2(treesize_ci)) / max_value
            usefulness[c] = usefulness_ci  # Armazena a utilidade calculada

        return usefulness

    def fit(self, X, y):
        """Treina o modelo Naive Bayes Hierárquico"""
        X = np.array(X, dtype=float)  # Converte os dados de entrada para array numpy
        y = np.array(y)  # Converte os rótulos para array numpy

        # Gera a hierarquia completa a partir das classes folha
        self.classes = sorted(list(set(y)))  # Obtém classes únicas ordenadas
        self.all_classes = self.gera_hierarquia_completa(self.classes)  # Gera hierarquia completa

        n_samples, n_features = X.shape  # Obtém número de amostras e características

        # Gera as relações hierárquicas
        self.ancestors = self.gera_ancestrais(self.all_classes)  # Gera ancestrais
        self.descendants = self.gera_descendentes(self.all_classes)  # Gera descendentes

        # Inicializa estruturas de dados
        self.feature_probs = [{} for _ in range(n_features)]  # Lista de dicionários para probabilidades dos atributos
        self.prior_prob = {classe: 0.0 for classe in self.all_classes}  # Inicializa probabilidades a priori

        # Conta valores distintos por atributo
        self.n_values_per_att = {
            i: len(np.unique(X[:, i][~np.isnan(X[:, i])])) for i in range(n_features)
        }  # Conta valores únicos excluindo NaN

        # Conta ocorrências de cada classe
        self.n_class_occurrences = Counter(y)  # Conta quantas vezes cada classe aparece

        # Para cada atributo
        for feature_idx in range(n_features):
            # Conta ocorrências de cada valor para cada classe
            for idx, c in enumerate(self.classes):
                # Seleciona instâncias da classe atual
                X_c = X[y == c]  # Filtra dados pela classe c

                if len(X_c) > 0:  # Se há instâncias desta classe
                    # Obtém contagens para cada valor
                    n_instances_class_c = X_c.shape[0]  # Número de instâncias da classe c
                    # Filtra valores NaN antes de contar
                    non_nan_indices = ~np.isnan(X_c[:, feature_idx])  # Índices dos valores não-NaN
                    valid_values = X_c[non_nan_indices, feature_idx]  # Valores válidos (não-NaN)

                    if len(valid_values) > 0:  # Se há valores válidos
                        feature_vals, counts = np.unique(valid_values, return_counts=True)  # Obtém valores únicos e suas contagens

                        # Calcula e armazena probabilidades para esta classe
                        feature_prob = {}  # Dicionário para probabilidades do atributo
                        for val, count in zip(feature_vals, counts):
                            # Converte para inteiro com verificação de NaN
                            val_int = int(val) if not np.isnan(val) else -999999  # Usa valor especial para NaN

                            # Calcula probabilidade com suavização de Laplace
                            prob = (count + self.alpha) / (n_instances_class_c + self.alpha * self.n_values_per_att[feature_idx])
                            feature_prob[val_int] = prob  # Armazena a probabilidade

                        self.feature_probs[feature_idx][c] = feature_prob  # Armazena probabilidades da classe

                        # Propaga contagens para os ancestrais
                        for ancestor in self.ancestors.get(c, []):
                            # Inicializa dicionário do ancestral se não existe
                            if ancestor not in self.feature_probs[feature_idx]:
                                self.feature_probs[feature_idx][ancestor] = {}

                            for val, count in zip(feature_vals, counts):
                                # Converte para inteiro com verificação de NaN
                                val_int = int(val) if not np.isnan(val) else -999999

                                # Inicializa contador se não existe
                                if val_int not in self.feature_probs[feature_idx][ancestor]:
                                    self.feature_probs[feature_idx][ancestor][val_int] = 0
                                # Adiciona contagens ponderadas ao ancestral
                                self.feature_probs[feature_idx][ancestor][val_int] += count

        # Normaliza probabilidades dos ancestrais
        for feature_idx in range(n_features):
            for ancestor in self.all_classes:
                if ancestor in self.feature_probs[feature_idx]:  # Se o ancestral tem dados
                    # Calcula total de instâncias para este ancestral
                    total_instances = 0
                    for val, count in self.feature_probs[feature_idx][ancestor].items():
                        total_instances += count  # Soma todas as contagens

                    # Normaliza probabilidades com suavização de Laplace
                    for val in self.feature_probs[feature_idx][ancestor]:
                        count = self.feature_probs[feature_idx][ancestor][val]  # Obtém contagem
                        # Aplica suavização de Laplace
                        self.feature_probs[feature_idx][ancestor][val] = (count + self.alpha) / (total_instances + self.alpha * self.n_values_per_att[feature_idx])

        # Calcula probabilidades a priori
        for c in self.classes:
            count = self.n_class_occurrences[c]  # Obtém contagem da classe
            # Calcula probabilidade a priori com suavização
            self.prior_prob[c] = (count + self.alpha) / (n_samples + self.alpha * len(self.classes))

            # Propaga para ancestrais
            for ancestor in self.ancestors.get(c, []):
                # Adiciona probabilidade da classe à probabilidade do ancestral
                self.prior_prob[ancestor] = self.prior_prob.get(ancestor, 0) + self.prior_prob[c]

        # Normaliza probabilidades a priori dos ancestrais
        for ancestor in self.all_classes:
            if ancestor in self.prior_prob:
                # Garante que a probabilidade não exceda 1.0
                self.prior_prob[ancestor] = min(self.prior_prob[ancestor], 1.0)

    def get_feature_probability(self, class_label, feature_idx, val):
        """Obtém probabilidade para um valor específico de atributo dada uma classe"""
        # Trata valores NaN
        if np.isnan(val):
            val_int = -999999  # Mesmo valor especial usado para NaN durante o treinamento
        else:
            val_int = int(val)  # Converte para inteiro

        # Busca direta se disponível
        if class_label in self.feature_probs[feature_idx] and val_int in self.feature_probs[feature_idx][class_label]:
            return self.feature_probs[feature_idx][class_label][val_int]  # Retorna probabilidade encontrada

        # Tenta usar probabilidades dos pais com backoff
        parts = class_label.split('.')  # Divide o nome da classe
        if len(parts) > 1:  # Se não é uma classe raiz
            parent = '.'.join(parts[:-1])  # Obtém classe pai
            # Verifica se o pai tem a probabilidade
            if parent in self.feature_probs[feature_idx] and val_int in self.feature_probs[feature_idx][parent]:
                # Usa probabilidade do pai com desconto
                return 0.1 * self.feature_probs[feature_idx][parent][val_int]

        # Fallback com suavização de Laplace
        return self.alpha / (self.n_class_occurrences.get(class_label, 0) + self.alpha * self.n_values_per_att[feature_idx])

    def predict(self, X_test, use_usefulness=False):
        """Prediz classes para instâncias de teste"""
        X_test = np.array(X_test, dtype=float)  # Converte dados de teste para array numpy
        predictions = []  # Lista para armazenar predições

        # Calcula utilidade se solicitado
        if use_usefulness:
            usefulness_dict = self.calculate_usefulness()  # Calcula dicionário de utilidade

        # Para cada instância de teste
        for x in X_test:
            posteriors = {}  # Dicionário para probabilidades posteriores

            # Calcula probabilidade posterior para cada classe
            for c in self.classes:  # Considera apenas classes folha para predição
                # Probabilidade a priori (em espaço logarítmico)
                log_prior = np.log(max(self.prior_prob.get(c, 1e-10), 1e-10))  # Evita log(0)

                # Calcula verossimilhança para cada atributo
                log_likelihood = 0
                for feature_idx, feature_val in enumerate(x):
                    prob = self.get_feature_probability(c, feature_idx, feature_val)  # Obtém probabilidade do atributo
                    log_likelihood += np.log(max(prob, 1e-10))  # Adiciona ao log da verossimilhança, evitando log(0)

                # Calcula posterior
                posterior = log_prior + log_likelihood  # Soma log das probabilidades

                # Aplica fator de utilidade se solicitado
                if use_usefulness:
                    usefulness_factor = usefulness_dict.get(c, 0.5)  # Obtém fator de utilidade
                    posterior += np.log(1.0 + usefulness_factor)  # Adiciona ao log da posterior

                posteriors[c] = posterior  # Armazena probabilidade posterior

            # Seleciona classe com maior probabilidade posterior
            if posteriors:  # Se há probabilidades calculadas
                best_class = max(posteriors.items(), key=lambda x: x[1])[0]  # Encontra classe com maior posterior
                predictions.append(best_class)  # Adiciona à lista de predições
            else:
                # Fallback
                predictions.append(self.classes[0])  # Usa primeira classe como padrão

        return np.array(predictions)  # Retorna array de predições

    def f_measure_hierarquica(self, predictions, y_true):
        """Calcula F-measure hierárquica"""
        numerador = 0  # Numerador para cálculo das métricas
        denominador_precision = 0  # Denominador para precisão
        denominador_recall = 0  # Denominador para recall

        # Para cada par de predição e valor verdadeiro
        for classe_predita, classe_verdadeira in zip(predictions, y_true):
            # Obtém ancestrais para classes predita e verdadeira
            if classe_predita in self.ancestors:
                # Lista de ancestrais incluindo a própria classe
                ancestrais_classe_predita = self.ancestors[classe_predita] + [classe_predita]
            else:
                ancestrais_classe_predita = [classe_predita]  # Apenas a própria classe

            if classe_verdadeira in self.ancestors:
                # Lista de ancestrais incluindo a própria classe
                ancestrais_classe_verdadeira = self.ancestors[classe_verdadeira] + [classe_verdadeira]
            else:
                ancestrais_classe_verdadeira = [classe_verdadeira]  # Apenas a própria classe

            # Conta ancestrais em comum
            classes_comum = len(set(ancestrais_classe_predita).intersection(set(ancestrais_classe_verdadeira)))

            numerador += classes_comum  # Adiciona ao numerador
            denominador_precision += len(ancestrais_classe_predita)  # Adiciona ao denominador da precisão
            denominador_recall += len(ancestrais_classe_verdadeira)  # Adiciona ao denominador do recall

        # Calcula métricas
        hierarchical_precision = numerador / max(denominador_precision, 1)  # Precisão hierárquica
        hierarchical_recall = numerador / max(denominador_recall, 1)  # Recall hierárquico
        # F-measure harmônica
        f_measure = (2 * hierarchical_precision * hierarchical_recall) / max((hierarchical_precision + hierarchical_recall), 1e-10)

        return hierarchical_precision, hierarchical_recall, f_measure

    def evaluate(self, y_true, y_pred):
        """Avalia performance do modelo usando métricas hierárquicas"""
        return self.f_measure_hierarquica(y_pred, y_true)  # Chama função de F-measure hierárquica

def main(arff_file_path):
    """Função principal para executar o experimento"""
    reader = ArffReader()  # Cria instância do leitor ARFF
    reader.read_arff(arff_file_path)  # Lê o arquivo ARFF

    # Converte dados para formato numérico, tratando valores vazios como NaN
    X = [list(map(lambda x: float(x) if x else float('nan'), row[:-1])) for row in reader.data]
    # print("Dados:")
    # for line in X:
    #   print(line)
    y = [row[-1] for row in reader.data]  # Extrai rótulos (última coluna)
    # print("Classes:")
    # print(y)


    print(f"\n=== Test avec alpha = {alpha} ===")  # Exibe valor de alpha sendo testado

    # Verificar erro aqui, pois não devemos dividir em treino e teste novamente (Parei aqui!)
    # Divide dados em conjuntos de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Treina e avalia o modelo
    model = HierarchicalNaiveBayes(alpha=alpha)  # Cria instância do modelo
    model.fit(X_train, y_train)  # Treina o modelo
    # Predição apenas no conjunto de teste
    y_pred = model.predict(X_test)  # Faz predições

    # Avalia com e sem fator de utilidade
    print("\n🔍 Résultats sans facteur d'utilité:")  # Exibe resultados sem fator de utilidade
    y_pred_no_usefulness = model.predict(X_test, use_usefulness=False)  # Prediz sem utilidade
    hP1, hR1, hF1 = model.evaluate(y_test, y_pred_no_usefulness)  # Avalia performance
    print(f"Précision hiérarchique (hP): {hP1 * 100:.2f}%")  # Exibe precisão hierárquica
    print(f"Rappel hiérarchique (hR): {hR1 * 100:.2f}%")  # Exibe recall hierárquico
    print(f"F1-score hiérarchique (hF): {hF1 * 100:.2f}%")  # Exibe F1-score hierárquico

if __name__ == "__main__":
    """Bloco principal de execução"""
    arff_file_path = "/content/sample_data/GPCR-PfamTES0.arff" # Caminho para o arquivo ARFF

    # main(arff_file_path)

    # Testa com diferentes valores de alpha
    for alpha in [0.1,0.5,1.0]:  # Lista de valores de alpha para testar
        main(arff_file_path)  # Executa experimento para cada valor de alpha


Total d'attributs : 76
Total d'instances : 653
Dados:
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0